In [1]:
%config IPCompleter.greedy=True

In [2]:
#this is a smart chatbot program

In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.


In [5]:
#import librearies

In [10]:
from newspaper import Article 
import random 
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
import pandas as pd
import os
warnings.filterwarnings('ignore')

In [11]:
#Download the punkt package

In [12]:
nltk.download('punkt', quiet=True)


True

In [ ]:
article = Article ('https://www.mayoclinic.org/es-es/diseases-conditions/coronavirus/symptoms-causes/syc-20479963')
article.download()
article.parse()
article.nlp()
corpus = article.text

In [ ]:
#mainpath = "../datasets/"
#filename = "210518COVID19MEXICO.csv"
#fullpath = os.path.join(mainpath, filename)

In [ ]:
#data = pd.read_csv(fullpath)

In [ ]:
#data.head()

In [ ]:
# definimos claves de campos 
hombre = 1 
mujer = 2
clave_entidad_nac = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
entidad_nac = ['aguascalientes', 'baja california', 'baja california sur', 'campeche', 'coahuila', 'colima', 'chiapas', 'chihuahua', 'cdmx', 'durango', 'guanajuato', 'guerrero', 'hidalgo', 'jalisco', 'estado de mexico', 'michoacan', 'morelos', 'nayarit', 'nuevo leon', 'oaxaca', 'puebla', 'queretaro', 'quintana roo', 'san luis potosi', 'sinaloa', 'sonora', 'sabasco', 'tamaulipas', 'tlaxcala', 'veracruz', 'yucatan', 'zacatecas']



In [ ]:
#calculando consultas probables que pregunten 
#el estado que más casos tiene --- lo calcularemos con el campo ENTIDAD_NAC
moda = int(data["ENTIDAD_NAC"].mode())
print(moda)
longitud = len(clave_entidad_nac)
if moda in clave_entidad_nac:
    posicion = clave_entidad_nac.index(moda)
    entidad = entidad_nac[posicion]
casos = (data["ENTIDAD_NAC"] == moda).values.ravel().sum()
print (f"La ciudad que ha tenido mayor número de casos confirmados es {entidad} con un total de {casos}")


In [ ]:
#print article text

In [ ]:
print(corpus)

In [ ]:
#tokenization

In [ ]:
text = corpus 
sentence_list = nltk.sent_tokenize(text) #a list of sentences

In [ ]:
print(sentence_list)

In [ ]:
#Function to return a random greeting response to the users 
def greeting_response(text):
    text = text.lower()
    
    #Bots greeting response 
    bot_greetings = ['Saludos', 'Hey', 'Hola!', 'Bienvenido', 'hola']
    #Users greeting 
    user_greetings = ['howdy', 'hi', 'hey', 'saludos', 'hola']
    for word in text.split():
        if word in user_greetings:
            return random.choice(bot_greetings)

In [ ]:
def index_sort(list_var): 
    length = len(list_var)
    list_index = list(range(0, length))
    x=list_var
    
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                temp = list_index[i]
                list_index[i]=list_index[j]
                list_index[j]=temp
    return list_index

In [ ]:
#create bots response 
def bot_response(user_input):
    user_input = user_input.lower()
    sentence_list.append(user_input)
    bot_response=''
    cm = CountVectorizer().fit_transform(sentence_list)
    similarity_scores = cosine_similarity(cm[-1], cm)
    similarity_scores_list = similarity_scores.flatten()
    index = index_sort(similarity_scores_list)
    index = index[1:]
    response_flag = 0
    
    j = 0
    for i in range(len(index)):
        if similarity_scores_list[index[i]] > 0.0: 
            bot_response = bot_response+' '+sentence_list[index[i]]
            response_flag = 1 
            j += 1
        if j > 2: 
            break
            
        if response_flag == 0:
            bot_response = bot_response+" lo siento, no encontré nada que pueda ayudarte."
            
        sentence_list.remove(user_input)
        
        return bot_response

In [ ]:
#start chat
print('Doc Bot: Soy el doctor bot. Estaré respondiendo tus preguntas acerca del COVID-19. Si quieres terminar de hablar conmigo, di chao')

exit_list = ['chao', 'bye', 'adios', 'salir']
while(True):
    user_input = input()
    if user_input.lower() in exit_list:
        print("Doc Bot: Nos vemos luego")
        break
    else: 
        if greeting_response(user_input) != None:
            print("Doc Bot: "+ greeting_response(user_input))
        else: 
            print('Doc Bot: '+ bot_response(user_input))